<a href="https://colab.research.google.com/github/LC1332/graphrag-practice-chinese/blob/main/notebook/build_graph_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 对于凡人修仙传的graphrag抽取和实践

项目地址: https://github.com/LC1332/graphrag-practice-chinese

- 李鲁鲁fork了这个项目，并且尝试将数据替换为凡人修仙传
- 原始的动机，是希望结合GraphRAG，每次询问一个修仙概念，然后建立修仙概念和现实世界自然科学的联系
- 逐步编写《修仙世界的自然原理》杂记
- 李鲁鲁把项目调整到可以在colab运行

# 在colab上配置环境

In [1]:
!git clone https://github.com/LC1332/graphrag-practice-chinese

Cloning into 'graphrag-practice-chinese'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 123 (delta 58), reused 89 (delta 29), pack-reused 0 (from 0)
Receiving objects: 100% (123/123), 2.93 MiB | 7.53 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [2]:
%cd ./graphrag-practice-chinese

/content/graphrag-practice-chinese


In [3]:
# 安装项目运行所需的依赖
!pip install -q -r ./requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.3/371.3 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.2/188.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 24.8 MB/s eta 0:00:00
   ━━━━━

In [4]:
# 创建 input 目录，用于构建索引的文本文件默认存放于该目录下，可以按需修改 settings.yaml 文件中的 input 部分来指定路径
!mkdir ./input


In [5]:
# 这一命令将在 graphrag-practice-chinese 目录中创建两个文件：.env 和 settings.yaml
!python -m graphrag.index --init --root ./

2025-02-24 01:42:53.165932: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740361373.191409     667 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740361373.199386     667 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Initializing project at ./
⠋ GraphRAG Indexer 

# 将GLM的key导入到.env文件

这里需要你的colab中配置了GLM_API_Key，在左侧点击小钥匙就可以配置。

In [6]:
from google.colab import userdata
glm_api_key = userdata.get('GLM_API_KEY')

In [7]:
# 将 GLM_API_KEY={glm_api_key}写入到.env
with open(".env", "w") as f:
  f.write(f"GLM_API_KEY={glm_api_key}")


# 用settings.demo.yaml覆盖settings.yaml

这里因为我GLM充了不少钱，所以模型我选用了GLM-4-Air，如果你没有充钱，或者钱比较少，可以去用免费的GLM-4-Flash。

另外我发现GLM-4-Air是可以使用json mode的，这里我把yaml里面的设置改为true了。

In [8]:
# prompt: 用settings.demo.yaml覆盖settings.yaml

import shutil
import os

# 覆盖 settings.yaml
if os.path.exists("./settings.demo.yaml"):
  shutil.copyfile("./settings.demo.yaml", "./settings.yaml")
  print("settings.yaml has been overwritten by settings.demo.yaml")
else:
  print("settings.demo.yaml not found, settings.yaml remains unchanged.")


settings.yaml has been overwritten by settings.demo.yaml


# 将输入文件复制到input

这里我选取到截止2025年3月接近动画片的外海剧情的小说前700章。

你可以在这里把input里面的文件替换为你自己想抽取GraphRAG的文件。

直觉来说，对小说进行合理的分章可以提升效果。

**初步做实验建议用fanren_20**来进行实验

这里我后面为了保险起见实际上是增量式逐步抽取的（我怕抽到一半给我error）

In [9]:
# prompt: 将lulu_exp/fanren_684.zip中的内容 直接解压到input中

import zipfile
import os

# 定义压缩文件路径和解压目标路径
zip_file_path = "lulu_exp/fanren_20.zip"
extract_path = "input"

# 创建解压目标目录，如果不存在
os.makedirs(extract_path, exist_ok=True)

# 打开压缩文件
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # 提取所有文件到目标目录
    zip_ref.extractall(extract_path)


# 进行抽取

我700章在colab上运行了大约一个小时

In [10]:
!python -m graphrag.index --root ./

2025-02-24 01:43:34.768128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740361414.813309     895 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740361414.825497     895 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🚀 Reading settings from settings.yaml
⠦ GraphRAG Indexer 
⠧ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 21 files loaded (0 filtered) ━━━━━━━━━ 100% 0:00:00 0:00:00
⠧ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 21 files loaded (0 filtered) ━━━━━━━━━ 100% 0:00:00 0:00:00
⠧ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 21 files loaded (0 filtered) ━━━━━━━━━ 100% 0:00:00 0:00:00
⠧ GraphRAG Index

# 将抽取结果保存到zip文件


这个zip我直接放在项目里面了，这样clone项目之后，是可以直接query信息的

这个我们会在一个inference的脚本里面去写

In [11]:
import zipfile
import os

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file),
                       os.path.relpath(os.path.join(root, file),
                                       os.path.join(path, '..')))

def create_zip():
    zipf = zipfile.ZipFile('fanren_output_20.zip', 'w', zipfile.ZIP_DEFLATED)
    zipdir('cache', zipf)
    zipdir('output', zipf)
    zipf.close()

create_zip()


# 尝试query信息

In [12]:
!python -m graphrag.query --root ./ --method global "灵气是什么？"

2025-02-24 01:46:50.717636: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740361610.741449    1716 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740361610.748733    1716 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO: Reading settings from settings.yaml
creating llm client with {'api_key': 'REDACTED,len=49', 'type': "openai_chat", 'model': 'glm-4-air', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 0.1, 'n': 1, 'request_timeout': 180.0, 'api_base': 'https://open.bigmodel.cn/api/paas/v4', 'api_version': None, 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': None, 'model_supports_json': True, 'to

---

# 下面都是分步提取

我分步提取主要是为了防止bug

In [14]:
# prompt: 将lulu_exp/fanren_684.zip中的内容 直接解压到input中

import zipfile
import os

# 定义压缩文件路径和解压目标路径
zip_file_path = "lulu_exp/fanren_20_100.zip"
extract_path = "input"

# 创建解压目标目录，如果不存在
os.makedirs(extract_path, exist_ok=True)

# 打开压缩文件
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # 提取所有文件到目标目录
    zip_ref.extractall(extract_path)


In [15]:
!python -m graphrag.index --root ./

流式输出内容被截断，只能显示最后 5000 行内容。
├── create_base_extracted_entities
├── create_summarized_entities
├── create_base_entity_graph
├── create_final_entities
├── create_final_nodes
├── create_final_communities
├── join_text_units_to_entity_ids
├── create_final_relationships
├── join_text_units_to_relationship_ids
└── create_final_community_reports
⠏ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 101 files loaded (0 filtered) ━━━━━━━━ 100% 0:00:00 0:00:00
├── create_base_text_units
├── create_base_extracted_entities
├── create_summarized_entities
├── create_base_entity_graph
├── create_final_entities
├── create_final_nodes
├── create_final_communities
├── join_text_units_to_entity_ids
├── create_final_relationships
├── join_text_units_to_relationship_ids
└── create_final_community_reports
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 101 files loaded (0 filtered) ━━━━━━━━ 100% 0:00:00 0:00:00
├── create_base_text_units
├── create_base_extracted_entities
├── create_summa

In [16]:
# prompt: 将lulu_exp/fanren_684.zip中的内容 直接解压到input中

import zipfile
import os

# 定义压缩文件路径和解压目标路径
zip_file_path = "lulu_exp/fanren_101_200.zip"
extract_path = "input"

# 创建解压目标目录，如果不存在
os.makedirs(extract_path, exist_ok=True)

# 打开压缩文件
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # 提取所有文件到目标目录
    zip_ref.extractall(extract_path)

!python -m graphrag.index --root ./


流式输出内容被截断，只能显示最后 5000 行内容。
├── create_base_entity_graph
├── create_final_entities
├── create_final_nodes
├── create_final_communities
├── join_text_units_to_entity_ids
├── create_final_relationships
├── join_text_units_to_relationship_ids
└── create_final_community_reports
⠧ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 201 files loaded (0 filtered) ━━━━━━━━ 100% 0:00:00 0:00:00
├── create_base_text_units
├── create_base_extracted_entities
├── create_summarized_entities
├── create_base_entity_graph
├── create_final_entities
├── create_final_nodes
├── create_final_communities
├── join_text_units_to_entity_ids
├── create_final_relationships
├── join_text_units_to_relationship_ids
└── create_final_community_reports
⠙ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 201 files loaded (0 filtered) ━━━━━━━━ 100% 0:00:00 0:00:00
├── create_base_text_units
├── create_base_extracted_entities
├── create_summarized_entities
├── create_base_entity_graph
├── create_final_entit

In [17]:
!python -m graphrag.query --root ./ --method global "灵气是什么？"

2025-02-24 02:18:20.325616: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740363500.353207    9334 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740363500.361436    9334 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO: Reading settings from settings.yaml
/usr/local/lib/python3.11/dist-packages/graphrag/query/indexer_adapters.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] =

In [18]:
# prompt: 将lulu_exp/fanren_684.zip中的内容 直接解压到input中

import zipfile
import os

# 定义压缩文件路径和解压目标路径
zip_file_path = "lulu_exp/fanren_201_300.zip"
extract_path = "input"

# 创建解压目标目录，如果不存在
os.makedirs(extract_path, exist_ok=True)

# 打开压缩文件
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # 提取所有文件到目标目录
    zip_ref.extractall(extract_path)

!python -m graphrag.index --root ./


流式输出内容被截断，只能显示最后 5000 行内容。
├── create_base_extracted_entities
├── create_summarized_entities
├── create_base_entity_graph
├── create_final_entities
├── create_final_nodes
├── create_final_communities
├── join_text_units_to_entity_ids
├── create_final_relationships
├── join_text_units_to_relationship_ids
└── create_final_community_reports
⠙ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 301 files loaded (0 filtered) ━━━━━━━━ 100% 0:00:00 0:00:00
├── create_base_text_units
├── create_base_extracted_entities
├── create_summarized_entities
├── create_base_entity_graph
├── create_final_entities
├── create_final_nodes
├── create_final_communities
├── join_text_units_to_entity_ids
├── create_final_relationships
├── join_text_units_to_relationship_ids
└── create_final_community_reports
⠸ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 301 files loaded (0 filtered) ━━━━━━━━ 100% 0:00:00 0:00:00
├── create_base_text_units
├── create_base_extracted_entities
├── create_summa

In [19]:
import zipfile
import os

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file),
                       os.path.relpath(os.path.join(root, file),
                                       os.path.join(path, '..')))

def create_zip():
    zipf = zipfile.ZipFile('fanren_output_20.zip', 'w', zipfile.ZIP_DEFLATED)
    zipdir('cache', zipf)
    zipdir('output', zipf)
    zipf.close()

create_zip()


In [20]:
# prompt: 将lulu_exp/fanren_684.zip中的内容 直接解压到input中

import zipfile
import os

# 定义压缩文件路径和解压目标路径
zip_file_path = "lulu_exp/fanren_301_400.zip"
extract_path = "input"

# 创建解压目标目录，如果不存在
os.makedirs(extract_path, exist_ok=True)

# 打开压缩文件
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # 提取所有文件到目标目录
    zip_ref.extractall(extract_path)

# !python -m graphrag.index --root ./


In [21]:
# prompt: 将lulu_exp/fanren_684.zip中的内容 直接解压到input中

import zipfile
import os

# 定义压缩文件路径和解压目标路径
zip_file_path = "lulu_exp/fanren_401_500.zip"
extract_path = "input"

# 创建解压目标目录，如果不存在
os.makedirs(extract_path, exist_ok=True)

# 打开压缩文件
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # 提取所有文件到目标目录
    zip_ref.extractall(extract_path)

!python -m graphrag.index --root ./


流式输出内容被截断，只能显示最后 5000 行内容。
├── create_summarized_entities
├── create_base_entity_graph
├── create_final_entities
├── create_final_nodes
├── create_final_communities
├── join_text_units_to_entity_ids
├── create_final_relationships
├── join_text_units_to_relationship_ids
└── create_final_community_reports
⠸ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 501 files loaded (0 filtered) ━━━━━━━━ 100% 0:00:00 0:00:00
├── create_base_text_units
├── create_base_extracted_entities
├── create_summarized_entities
├── create_base_entity_graph
├── create_final_entities
├── create_final_nodes
├── create_final_communities
├── join_text_units_to_entity_ids
├── create_final_relationships
├── join_text_units_to_relationship_ids
└── create_final_community_reports
⠸ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 501 files loaded (0 filtered) ━━━━━━━━ 100% 0:00:00 0:00:00
├── create_base_text_units
├── create_base_extracted_entities
├── create_summarized_entities
├── create_base_enti

In [22]:
import zipfile
import os

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file),
                       os.path.relpath(os.path.join(root, file),
                                       os.path.join(path, '..')))

def create_zip():
    zipf = zipfile.ZipFile('fanren_output_500.zip', 'w', zipfile.ZIP_DEFLATED)
    zipdir('cache', zipf)
    zipdir('output', zipf)
    zipf.close()

create_zip()


In [29]:
# prompt: 删除input文件夹中, 序号比0500_book.txt更大的txt

import os

def remove_larger_files(directory, threshold_filename="0500_book.txt"):
    """Removes .txt files in the given directory with a numerical index larger than the threshold file.

    Args:
        directory: The directory containing the .txt files.
        threshold_filename: The filename that defines the threshold index.
    """

    threshold_index = int(threshold_filename.split("_")[0])
    remove_count = 0
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            try:
                file_index = int(filename.split("_")[0])
                if file_index > threshold_index:
                    file_path = os.path.join(directory, filename)
                    os.remove(file_path)
                    remove_count += 1
                    # print(f"Removed file: {filename}")
            except (ValueError, IndexError):
                print(f"Skipping file (invalid format): {filename}")

    print(remove_count," removed")

# Example usage (replace with your actual directory)
remove_larger_files("./input")


150  removed


In [30]:
# prompt: 将lulu_exp/fanren_684.zip中的内容 直接解压到input中

import zipfile
import os

# 定义压缩文件路径和解压目标路径
zip_file_path = "lulu_exp/fanren_501_600.zip"
extract_path = "input"

# 创建解压目标目录，如果不存在
os.makedirs(extract_path, exist_ok=True)

# 打开压缩文件
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # 提取所有文件到目标目录
    zip_ref.extractall(extract_path)

!python -m graphrag.index --root ./


流式输出内容被截断，只能显示最后 5000 行内容。
├── create_final_nodes
├── create_final_communities
├── join_text_units_to_entity_ids
├── create_final_relationships
├── join_text_units_to_relationship_ids
└── create_final_community_reports
⠦ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 601 files loaded (0 filtered) ━━━━━━━━ 100% 0:00:00 0:00:00
├── create_base_text_units
├── create_base_extracted_entities
├── create_summarized_entities
├── create_base_entity_graph
├── create_final_entities
├── create_final_nodes
├── create_final_communities
├── join_text_units_to_entity_ids
├── create_final_relationships
├── join_text_units_to_relationship_ids
└── create_final_community_reports
⠏ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 601 files loaded (0 filtered) ━━━━━━━━ 100% 0:00:00 0:00:00
├── create_base_text_units
├── create_base_extracted_entities
├── create_summarized_entities
├── create_base_entity_graph
├── create_final_entities
├── create_final_nodes
├── create_final_communities

In [31]:
!python -m graphrag.query --root ./ --method global "灵气是什么？"

2025-02-24 04:17:48.032051: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740370668.088639   38081 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740370668.106060   38081 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO: Reading settings from settings.yaml
/usr/local/lib/python3.11/dist-packages/graphrag/query/indexer_adapters.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] =

In [32]:
import zipfile
import os

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file),
                       os.path.relpath(os.path.join(root, file),
                                       os.path.join(path, '..')))

def create_zip():
    zipf = zipfile.ZipFile('fanren_output_600.zip', 'w', zipfile.ZIP_DEFLATED)
    zipdir('cache', zipf)
    zipdir('output', zipf)
    zipf.close()

create_zip()


In [24]:
# prompt: 将lulu_exp/fanren_684.zip中的内容 直接解压到input中

import zipfile
import os

# 定义压缩文件路径和解压目标路径
zip_file_path = "lulu_exp/fanren_601_700.zip"
extract_path = "input"

# 创建解压目标目录，如果不存在
os.makedirs(extract_path, exist_ok=True)

# 打开压缩文件
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # 提取所有文件到目标目录
    zip_ref.extractall(extract_path)

!python -m graphrag.index --root ./


流式输出内容被截断，只能显示最后 5000 行内容。
└── create_summarized_entities
⠧ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 701 files loaded (0 filtered) ━━━━━━━━ 100% 0:00:00 0:00:00
├── create_base_text_units
├── create_base_extracted_entities
└── create_summarized_entities
⠼ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 701 files loaded (0 filtered) ━━━━━━━━ 100% 0:00:00 0:00:00
├── create_base_text_units
├── create_base_extracted_entities
└── create_summarized_entities
⠼ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 701 files loaded (0 filtered) ━━━━━━━━ 100% 0:00:00 0:00:00
├── create_base_text_units
├── create_base_extracted_entities
└── create_summarized_entities
⠦ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 701 files loaded (0 filtered) ━━━━━━━━ 100% 0:00:00 0:00:00
├── create_base_text_units
├── create_base_extracted_entities
└── create_summarized_entities
⠏ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 701 files loaded (0 filte

In [25]:
import zipfile
import os

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file),
                       os.path.relpath(os.path.join(root, file),
                                       os.path.join(path, '..')))

def create_zip():
    zipf = zipfile.ZipFile('fanren_output_all.zip', 'w', zipfile.ZIP_DEFLATED)
    zipdir('cache', zipf)
    zipdir('output', zipf)
    zipf.close()

create_zip()
